In [1]:
import torch
import torchvision
import torchvision.transforms as transfroms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
batch_size = 10
transfrom = transfroms.Compose([
    transfroms.ToTensor(),
    transfroms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

In [3]:
trainset = torchvision.datasets.CIFAR10(root="./data",train=True, download=True,  transform=transfrom)
trainloder = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

100.0%


In [4]:
testset = torchvision.datasets.CIFAR10(root="./data",train=False, download=True,  transform=transfrom)
testloder = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)